In [12]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests

### Foodmart product sales

In [2]:
sales = pd.read_csv('Data/foodmart.sales.tsv', 
                     sep='\t', 
                     header=0,
                     parse_dates=[2])

In [3]:
sales.head()

,product_id,store_id,date,sales
0,4,6,1997-01-01,4
1,25,6,1997-01-01,3
2,48,6,1997-01-01,3
3,76,6,1997-01-01,4
4,119,6,1997-01-01,3


In [5]:
products = pd.read_csv('Data/foodmart.products.tsv',
                       sep='\t',
                       header=0)

In [6]:
products.head()

,product_class_id,product_id,brand_name,product_name,SKU,SRP,gross_weight,net_weight,recyclable_package,low_fat,units_per_case,cases_per_pallet,shelf_width,shelf_height,shelf_depth
0,30,1,Washington,Washington Berry Juice,90748583674,2.85,8.39,6.39,False,False,30,14,16.9,12.60,7.40
1,52,2,Washington,Washington Mango Drink,96516502499,0.74,7.42,4.42,False,True,18,8,13.4,3.71,22.60
2,52,3,Washington,Washington Strawberry Drink,58427771925,0.83,13.10,11.10,True,True,17,13,14.4,11.00,7.77
3,19,4,Washington,Washington Cream Soda,64412155747,3.64,10.60,9.60,True,False,26,10,22.9,18.90,7.93
4,19,5,Washington,Washington Diet Soda,85561191439,2.19,6.66,4.65,True,False,7,10,20.7,21.90,19.20


In [7]:
sales = sales.merge(products[['product_id', 'product_name']],
                    on = ['product_id'], how='inner')

In [8]:
sales.head()

,product_id,store_id,date,sales,product_name
0,4,6,1997-01-01,4,Washington Cream Soda
1,4,7,1997-01-05,3,Washington Cream Soda
2,4,6,1997-01-06,2,Washington Cream Soda
3,4,17,1997-01-11,2,Washington Cream Soda
4,4,24,1997-01-11,2,Washington Cream Soda


### Корреляция Пирсона

In [9]:
sparse_sales = pd.pivot_table(sales, values='sales', index=['date', 'store_id'],
                              columns=['product_name'], fill_value=0, aggfunc=lambda x: x)

In [11]:
sparse_sales.head()

product_name         ADJ Rosy Sunglasses             ...               Washington Strawberry Drink
date       store_id                                  ...                                          
1997-01-01 6                           0             ...                                         0
           14                          0             ...                                         0
1997-01-02 11                          0             ...                                         0
           23                          0             ...                                         0
1997-01-03 7                           0             ...                                         0

[5 rows x 1559 columns]

In [14]:
corr_data = []
for i, lhs_column in tqdm_notebook(enumerate(sparse_sales.columns), total=sparse_sales.shape[1]/2):
    for j, rhs_column in enumerate(sparse_sales.columns):
        if i >= j:
            continue
        
        corr, p = pearsonr(sparse_sales[lhs_column], sparse_sales[rhs_column])
        corr_data.append([lhs_column, rhs_column, corr, p])

In [16]:
sales_correlation = pd.DataFrame.from_records(corr_data)
sales_correlation.columns=['Product_A', 'Product_B', 'corr', 'p']

In [17]:
sales_correlation.head()

,Product_A,Product_B,corr,p
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484


Количество отвергаемых гипотез без поправки на множественную проверку

In [18]:
(sales_correlation['p'] < 0.05).value_counts()

False    982453
True     232008
Name: p, dtype: int64

## Поправка на множественную проверку

### Метод Холма

In [19]:
reject, p_corrected, a1, a2 = multipletests(sales_correlation['p'],
                                            alpha=0.05,
                                            method='holm')

In [20]:
sales_correlation['p_corrected'] = p_corrected
sales_correlation['reject'] = reject

In [22]:
sales_correlation.head()

,Product_A,Product_B,corr,p,p_corrected,reject
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414,1.0,False
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396,1.0,False
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546,1.0,False
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529,1.0,False
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484,1.0,False


In [23]:
sales_correlation['reject'].value_counts()

False    1212733
True        1728
Name: reject, dtype: int64

### Метод Бенджамини - Хохберга

In [24]:
reject, p_corrected, a1, a2 = multipletests(sales_correlation['p'],
                                            alpha=0.05,
                                            method='fdr_bh')

In [25]:
sales_correlation['p_corrected'] = p_corrected
sales_correlation['reject'] = reject

In [26]:
sales_correlation.head()

,Product_A,Product_B,corr,p,p_corrected,reject
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414,0.203716,False
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396,0.956078,False
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546,0.630699,False
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529,0.600790,False
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484,0.541916,False


In [27]:
sales_correlation['reject'].value_counts()

False    1138407
True       76054
Name: reject, dtype: int64